# Сборный проект - 2

## Описание проекта

Вы работаете в стартапе, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи вашего мобильного приложения. Предобработайте данные. Изучите воронку продаж. Как пользователи доходят до покупки? Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно? **Исследуйте результаты A/A/B-эксперимента**. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Какой шрифт лучше?

## Структура проекта

1. [Знакомство с данными](#1)
2. [Предобработка данных](#2)
3. [Изучаем и проверяем данные](#3)
4. [Воронка событий](#4)
 - [4.1. Количество логов и пользователей по событиям](#4.1)
 - [4.2. Порядок событий и воронка. Какая доля доходит до покупки?](#4.2)

5. [Анализ A/A/B-теста](#5)
 - [5.1. Количество пользователей в группах](#5.1)
 - [5.2. A/A тест](#5.2)
 - [5.3. Тестирование контрольных групп (246, 247) с экспериментальной (248)](#5.3)
 - [5.4. Выбранный критерий значимости](#5.4)
6. [Результаты A/A/B-теста](#6)


## Прекод

In [1]:
!pip install plotly -U

In [2]:
# игнорируем предупреждения
import warnings
warnings.filterwarnings('ignore')

# работа с таблицамими, вычисления
import numpy as np
import pandas as pd
from scipy import stats as st

# визуализация
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
from plotly import graph_objects as go

# параметры визуализации
sns.set()
sns.set_context(
    "notebook", 
    font_scale=1.3,       
    rc={ 
        "figure.figsize": (12, 10), 
        "axes.titlesize": 18 
    }
)
from matplotlib import rcParams
rcParams['figure.figsize'] = 12, 10

# параметры отображения таблиц
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## 1. Знакомство с данными <a name="1"></a>

In [3]:
LINK = ''
PRAKTIKUM_DIR = '/datasets/'

LOGS_DATA = 'logs_exp.csv'

In [4]:
try:
    raw_df = pd.read_csv(PRAKTIKUM_DIR+LOGS_DATA, sep='\t')
    print('Файлы прочитаны. Работаем в среде Я.Практикума')
except:
    raw_df = pd.read_csv(LINK+LOGS_DATA, sep='\t')
    print('Данные прочитаны. Работаем вне среды Я.Практикума')

Данные прочитаны. Работаем вне среды Я.Практикума


In [5]:
print('Размерность raw_df:', raw_df.shape)

Размерность raw_df: (244126, 4)


In [6]:
raw_df.head()

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


**Описание столбцов:**
- EventName — название события;
- DeviceIDHash — уникальный идентификатор пользователя;
- EventTimestamp — время события;
- ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

In [7]:
raw_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 22.9 MB


Посмотрим на распределения событий и групп тестирования.

In [8]:
raw_df['EventName'].value_counts()

MainScreenAppear           119205
OffersScreenAppear          46825
CartScreenAppear            42731
PaymentScreenSuccessful     34313
Tutorial                     1052
Name: EventName, dtype: int64

In [9]:
raw_df['ExpId'].value_counts()

248    85747
246    80304
247    78075
Name: ExpId, dtype: int64

Посмотрим не только на полные, но и скрытые дубликаты.

In [10]:
print('Количество полных дубликатов:', raw_df.duplicated().sum())
print('Количество дубликатов без столбца группы ExpId:', raw_df.iloc[:, :-1].duplicated().sum())
print('Количество дубликатов без столбцов EventName:', raw_df.iloc[:, 1:].duplicated().sum())


Количество полных дубликатов: 413
Количество дубликатов без столбца группы ExpId: 413
Количество дубликатов без столбцов EventName: 23569


In [11]:
raw_df[raw_df.iloc[:, 1:].duplicated(keep=False)].head(10)

,EventName,DeviceIDHash,EventTimestamp,ExpId
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
27,CartScreenAppear,2029140728621466572,1564160801,246
28,PaymentScreenSuccessful,2029140728621466572,1564160801,246
35,PaymentScreenSuccessful,746235405686708560,1564170266,248
36,CartScreenAppear,746235405686708560,1564170266,248
37,CartScreenAppear,6805714022805866600,1564170267,246
38,PaymentScreenSuccessful,6805714022805866600,1564170267,246
42,PaymentScreenSuccessful,6075091495307260046,1564209665,246
43,CartScreenAppear,6075091495307260046,1564209665,246


In [12]:
raw_df[raw_df.iloc[:, 1:].duplicated(keep=False)]['EventName'].value_counts()

CartScreenAppear           19991
PaymentScreenSuccessful    17775
MainScreenAppear            5618
OffersScreenAppear           487
Tutorial                      57
Name: EventName, dtype: int64

- имеем полные дубликаты. Избавимся от них. Их количество совпадает с дубликатами без столбца `ExpId` - значит, у нас нет событий, которые одновременно попали в несколько групп тестирования.
- имеем 23569 дубликатов без столбца события `event`. Больше всего одновременных событий приходится на события оплаты `PaymentScreenSuccessful` и корзины `CartScreenAppear`. Может быть, сразу после покупки нас кидает в корзину? По остальным подобным дубликатам ситуация не ясна. Не помешает уточнить у разработчиков и написать баг-репорт по мере необходимости.

Еще раз более явно убедимся в том, что ни один пользователь на попал более чем в 1 экспериментальную группу `exp_id`

In [13]:
raw_df.groupby('DeviceIDHash')['ExpId'].nunique().sort_values(ascending=True)

DeviceIDHash
6888746892508752       1
6216080220799726690    1
6215559225876063378    1
6215162890135937308    1
6213626876710715478    1
                      ..
3170212200647575044    1
3167974726645136146    1
3167390091686880227    1
3184294769293286652    1
9222603179720523844    1
Name: ExpId, Length: 7551, dtype: int64

Сплит-система отработала корректно. На каждого пользователя приходится только по 1-й группе.

### Промежуточный итог

- поменяем названия столбца на удобные, формат приведем к snake_case;
- изменим тип данных `EventTimestamp` с `int` на `datetime64[s]` (дата и время с точностью до секунд). Текущие значения выглядят как unix-время;
- пропусков в данных нет;
- есть 413 полных дубликатов-строк. Избавимся от них. Не ясна природа дубликатов без столбца событий `EventName`;
- можно немного оптимизировать память посредством даункастинга столбцов с числовым типом: `int64` -> `uint`, а также значительно оптимизировать посредством изменения столбца `event` с `object` -> `category`;
- убедились, что сплит-система корректно поместила каждого пользователя в одну группу.

## 2. Предобработка данных <a name="2"></a>

Оставим сырые данные в `raw_df`, а изменения будем вносить в `df`

In [14]:
df = raw_df.copy()

Меняем названия на более удобные.

In [15]:
df.columns = ['event', 'uid', 'event_ts', 'exp_id']

Меняем столбец `event_ts` из unix-формата времени в `datetime64[s]` (дата и время с точностью до секунд).

In [16]:
df['event_ts'] = df['event_ts'].astype('datetime64[s]')

In [17]:
df.head()

,event,uid,event_ts,exp_id
0,MainScreenAppear,4575588528974610257,2019-07-25 04:43:36,246
1,MainScreenAppear,7416695313311560658,2019-07-25 11:11:42,246
2,PaymentScreenSuccessful,3518123091307005509,2019-07-25 11:28:47,248
3,CartScreenAppear,3518123091307005509,2019-07-25 11:28:47,248
4,PaymentScreenSuccessful,6217807653094995999,2019-07-25 11:48:42,248


In [18]:
int_cols = df.select_dtypes(include='number').columns.tolist()

In [19]:
for col in int_cols:
  print('Тип данных столбца', col, 'изменен с', df[col].dtype, end='')
  df[col] = pd.to_numeric(df[col], downcast='unsigned')
  print(' на', df[col].dtype)

Тип данных столбца uid изменен с int64 на uint64
Тип данных столбца exp_id изменен с int64 на uint8


Наибольшее сокращение используемое памяти  даст преобразование `object` столбца `event` в тип данных `category`.

In [20]:
df['event'] = df['event'].astype('category')

In [21]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   event     244126 non-null  category      
 1   uid       244126 non-null  uint64        
 2   event_ts  244126 non-null  datetime64[ns]
 3   exp_id    244126 non-null  uint8         
dtypes: category(1), datetime64[ns](1), uint64(1), uint8(1)
memory usage: 4.2 MB


Использование памяти сократилось на ~82% — с 22.9 до 4.2 MB.

Удаляем явные дубликаты.

In [22]:
df = df.drop_duplicates()

Наконец, добавим столбец с датой без времени.

In [23]:
df['event_dt'] = df['event_ts'].astype('datetime64[D]')

In [24]:
df.sample(5, random_state=42)

,event,uid,event_ts,exp_id,event_dt
240000,CartScreenAppear,2570971451355530886,2019-08-07 18:04:22,248,2019-08-07
233810,CartScreenAppear,2431455045762717569,2019-08-07 15:20:38,247,2019-08-07
180426,MainScreenAppear,8233040176393470157,2019-08-06 05:28:15,246,2019-08-06
12575,MainScreenAppear,438481114985016111,2019-08-01 09:39:31,247,2019-08-01
201095,CartScreenAppear,4877161838950700944,2019-08-06 15:36:16,247,2019-08-06


## 3. Изучаем и проверяем данные <a name="3"></a>

Согласно заданию, проверим: 
- Сколько всего событий в логе?
- Сколько всего пользователей в логе?
- Сколько в среднем событий приходится на пользователя? Найдите максимальную и минимальную дату. Постройте гистограмму по дате и времени. Можно ли быть уверенным, что у вас одинаково полные данные за весь период? Технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные». Определите, с какого момента данные полные и отбросьте более старые. Данными за какой период времени вы располагаете на самом деле?
- Данными за какой период вы располагаете?
- Много ли событий и пользователей вы потеряли, отбросив старые данные?
- Проверьте, что у вас есть пользователи из всех трёх экспериментальных групп.

Идем по порядку.

In [25]:
df.head()

,event,uid,event_ts,exp_id,event_dt
0,MainScreenAppear,4575588528974610257,2019-07-25 04:43:36,246,2019-07-25
1,MainScreenAppear,7416695313311560658,2019-07-25 11:11:42,246,2019-07-25
2,PaymentScreenSuccessful,3518123091307005509,2019-07-25 11:28:47,248,2019-07-25
3,CartScreenAppear,3518123091307005509,2019-07-25 11:28:47,248,2019-07-25
4,PaymentScreenSuccessful,6217807653094995999,2019-07-25 11:48:42,248,2019-07-25


In [26]:
df.shape[0]

243713

**В логах 243713 событий.**

In [27]:
df['uid'].nunique()

7551

**В логах 7551 уникальных пользователей.**

In [28]:
df.shape[0] / len(df['uid'].unique())

32.27559263673685

**В среднем на пользователя в наших данных приходится ~32 события.**

In [29]:
print('Минимальная дата в логах:', df['event_dt'].min())
print('Максимальная дата в логах:', df['event_dt'].max())

Минимальная дата в логах: 2019-07-25 00:00:00
Максимальная дата в логах: 2019-08-07 00:00:00


In [30]:
event_dist = (
    df
    .groupby(['event_dt', 'exp_id'])
    .agg({'uid':'count'})
    .reset_index()
    .rename(columns={'uid':'n_events'})
    )

In [31]:
event_dist.head()

,event_dt,exp_id,n_events
0,2019-07-25,246,4
1,2019-07-25,247,1
2,2019-07-25,248,4
3,2019-07-26,246,14
4,2019-07-26,247,8


In [32]:
px.bar(
    event_dist, 
    x='event_dt',
    y='n_events', 
    color='exp_id',
    title='Динамика событий по экпериментальным группам',
    barmode='group')

- **лишь начиная с 1 августа эксперимент запущен в полную силу**. Анализ будем проводить с этой даты;
- похоже, что с 25 по 31 июля происходила подготовка к эксперименту: в экспериментальные группы попадала лишь небольшая часть трафика;
- визуально убедились, что **в каждой группе есть события**, в каждой дате события по группам распределены достаточно равномерно (в рамках адекватного разброса).


In [33]:
df.groupby('exp_id').agg({'uid':'nunique'})

,uid
exp_id,
246,2489
247,2520
248,2542


Также убедились, что в каждой группе есть пользователи, которые распределены достаточно равномерно. Хотя относительная разница между двумя контрольными группами 2520 / 2489 = ~1.2%. Стоит разобраться, почему сплит-система дает разницу в количестве пользователей более 1%.

In [34]:
df.query('event_dt >= "2019-08-01"').shape[0] / df.shape[0]

0.988404393692581

**Около 1.2% логов приходится на даты до фактической даты запуска эксперимента**. Исключаем эти строки.

In [35]:
df = df.query('event_dt >= "2019-08-01"')

## 4. Воронка событий <a name="4"></a>

Согласно заданиям:
- Посмотрите, какие события есть в логах, как часто они встречаются. Отсортируйте события по частоте.
- Посчитайте, сколько пользователей совершали каждое из этих событий. Отсортируйте события по числу пользователей. Посчитайте долю пользователей, которые хоть раз совершали событие.
- Предположите, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не нужно учитывать при расчёте воронки.
- По воронке событий посчитайте, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C посчитайте отношение числа пользователей с событием B к количеству пользователей с событием A, а также отношение числа пользователей с событием C к количеству пользователей с событием B.
- На каком шаге теряете больше всего пользователей?
- Какая доля пользователей доходит от первого события до оплаты?

Идем по порядку.

### 4.1. Количество логов и пользователей по событиям <a name="4.1"></a>

In [36]:
df.shape[0]

240887

In [37]:
event_report = (
    df
    .groupby('event', as_index=False)
    .agg({'event_ts':'count','uid':'nunique'})
    .rename(columns={'event_ts':'n_events', 'uid':'n_users'})
    .sort_values(by='n_users', ascending=False)
    )

In [38]:
df['uid'].nunique()

7534

In [39]:
event_report['share_of_users'] = (event_report['n_users'] / df['uid'].nunique()).round(3)

In [40]:
event_report.head()

,event,n_events,n_users,share_of_users
1,MainScreenAppear,117328,7419,0.985
2,OffersScreenAppear,46333,4593,0.610
0,CartScreenAppear,42303,3734,0.496
3,PaymentScreenSuccessful,33918,3539,0.470
4,Tutorial,1005,840,0.111


In [41]:
event_report.head()

,event,n_events,n_users,share_of_users
1,MainScreenAppear,117328,7419,0.985
2,OffersScreenAppear,46333,4593,0.610
0,CartScreenAppear,42303,3734,0.496
3,PaymentScreenSuccessful,33918,3539,0.470
4,Tutorial,1005,840,0.111


In [42]:
fig = px.bar(
    event_report, 
    x='event', 
    y='n_events', 
    color='event', 
    title='Распределение количества логов по событиям'
    )

fig.update_layout(showlegend=False)

**Наиболее распространенным событием по логам закономерно является страничка главного экрана `MainScreenAppear`, наименее распространенным — `Tutorial`**.

In [43]:
fig = px.bar(
    event_report, 
    x='event', 
    y='n_users', 
    color='event', 
    title='Распределение количества пользователей по событиям'
    )

fig.update_layout(showlegend=False)

**Распределение событий по количеству пользователей повторяет порядок из прошлого графика.** 

- от прошлого графика отличается событие Tutorial: оно имеет крайне низкую долю от логов, однако приличную долю по пользователям. Косвенно это говорит о том, что пользователи в среднем просто не смотрят туториал больше 1 раза в отличии от других событий, которые естественным образом повторяются много раз;
- уже вырисовывается предварительная воронка событий: от главного экрана до оплаты;

### 4.2. Порядок событий и воронка. Какая доля доходит до покупки? <a name="4.2"></a>

Условным порядок событий можно признать порядок событий по убыванию количества пользователей (график и таблица выше).

In [44]:
event_report

,event,n_events,n_users,share_of_users
1,MainScreenAppear,117328,7419,0.985
2,OffersScreenAppear,46333,4593,0.610
0,CartScreenAppear,42303,3734,0.496
3,PaymentScreenSuccessful,33918,3539,0.470
4,Tutorial,1005,840,0.111


**Наиболее разумный порядок событий: MainScreenAppear -> OffersScreenAppear -> CartScreenAppear -> PaymentScreenSuccessful**. Событие Tutorial при построении воронки лучше исключить, т.к. оно специфическое и может быть как первым событием для пользователя, так и где-то по середине и наиболее редко в конце. 

In [45]:
fig = go.Figure(
    go.Funnel(
        x=event_report.iloc[:-1]['n_users'], 
        y=event_report.iloc[:-1]['event'],
        textinfo = "value+percent initial",
    )
)

fig.update_layout(title={'text': 'Воронка событий (процент от первого этапа)'})

**От появления главного экрана до покупки доходит 48% пользователей.** 

Посмотрим, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем).


In [46]:
fig = go.Figure(
    go.Funnel(
        x=event_report.iloc[:-1]['n_users'], 
        y=event_report.iloc[:-1]['event'],
        textinfo = "value+percent previous",
    )
)

fig.update_layout(title={'text': 'Воронка событий (процент от предыдущего этапа)'})

**Наибольший отсев происходит после события `MainScreenAppear`, лишь 62% пользователей после этого события увидели следующий этап `OffersScreenAppear`, то есть 38% отсеились.**

- далее имеем 19% отсева с этапа `OffersScreenAppear` -> `CartScreenAppear`;
- удивительно, насколько высока конверсия из экрана корзины в покупки (всего 5% отсева). Либо наша корзина и всякого рода push-рассылка с напоминанием о покупках настроены почти идеально, либо наблюдаем такие значения из-за того, что **в логах эти два события часто происходят одновременно**. Возможно, после покупки по логам нас сразу кидает обратно в корзину. Стоит уточнить у разработчиков, т.к. **это может сильно искажать воронку и тесты**.

Дополнительно проверим, для какой доли пользователей какие события были первыми, а какие последними.

In [47]:
first_event = (
    df
    .groupby('uid', as_index=False)
    .agg({'event':'first'})
    .groupby('event', as_index=False)
    .count()
    .sort_values(by='uid', ascending=False)
    .rename(columns={'uid':'n_users_as_first_event'})
    )

last_event = (
    df
    .groupby('uid', as_index=False)
    .agg({'event':'last'})
    .groupby('event', as_index=False)
    .count()
    .sort_values(by='uid', ascending=False)
    .rename(columns={'uid':'n_users_as_last_event'})
    )

In [48]:
first_event

,event,n_users_as_first_event
1,MainScreenAppear,6305
4,Tutorial,783
2,OffersScreenAppear,288
3,PaymentScreenSuccessful,96
0,CartScreenAppear,62


In [49]:
event_report = (
    event_report
    .merge(first_event, on='event')
    .merge(last_event, on='event')
    )

In [50]:
event_report['share_as_first_event'] = (event_report['n_users_as_first_event'] / df['uid'].nunique()).round(3)
event_report['share_as_last_event'] = (event_report['n_users_as_last_event'] / df['uid'].nunique()).round(3)

In [51]:
event_report.columns.tolist()

['event',
 'n_events',
 'n_users',
 'share_of_users',
 'n_users_as_first_event',
 'n_users_as_last_event',
 'share_as_first_event',
 'share_as_last_event']

In [52]:
event_report = event_report[
    [
        'event',
        'n_events',
        'n_users',
        'share_of_users',
        'n_users_as_first_event',
        'share_as_first_event',
        'n_users_as_last_event',
        'share_as_last_event'
    ]
]

In [53]:
event_report[
    [
        'event', 
        'n_users_as_first_event',
        'share_as_first_event',
        'n_users_as_last_event',
        'share_as_last_event'
        ]
    ]

,event,n_users_as_first_event,share_as_first_event,n_users_as_last_event,share_as_last_event
0,MainScreenAppear,6305,0.837,4371,0.580
1,OffersScreenAppear,288,0.038,2394,0.318
2,CartScreenAppear,62,0.008,543,0.072
3,PaymentScreenSuccessful,96,0.013,222,0.029
4,Tutorial,783,0.104,4,0.001


- для почти ~10% пользователей первым событием был Tutorial, а последним событием он стал всего для 0.1%. Иными словами, люди почти всегда продолжают активность после просмотра туториала. Возможно, туториал оказался крайне полезным для пользователей и следует увеличить его охват; 
- каким-то образом для 96 пользователей первым залогированным событием стала сразу покупка. Стоит уточнить у коллег и написать баг-репорт по необходимости. Также контр-интуитивным событием выглядит возикновение экрана корзины как первое событие пользователя;
- Большинство пользователей отваливается в самом начале пути. На первые два этапа `MainScreenAppear` и `OffersScreenAppear` приходится около 88% первых событий пользователя и около 90% последних. Вполне закономерно. Чаще всего отсев происходит на этапе знакомства с продуктом.

## 5. Анализ A/A/B-теста <a name="5"></a>

- Сколько пользователей в каждой экспериментальной группе?
- Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверьте, находят ли статистические критерии разницу между выборками 246 и 247.
- Выберите самое популярное событие. Посчитайте число пользователей, совершивших это событие в каждой из контрольных групп. Посчитайте долю пользователей, совершивших это событие. Проверьте, будет ли отличие между группами статистически достоверным. Проделайте то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?
- Аналогично поступите с группой с изменённым шрифтом. Сравните результаты с каждой из контрольных групп в отдельности по каждому событию. Сравните результаты с объединённой контрольной группой. Какие выводы из эксперимента можно сделать?
- Какой уровень значимости вы выбрали при проверке статистических гипотез выше? Посчитайте, сколько проверок статистических гипотез вы сделали. При уровне значимости 0.1 каждый десятый раз можно получать ложный результат. Какой уровень значимости стоит применить? Если вы хотите изменить его, проделайте предыдущие пункты и проверьте свои выводы.

### 5.1. Количество пользователей в группах <a name="5.1"></a>

**Еще раз найдем пользователей по группам.** Для более удобно доступа к этим значениям из фрейма сделаем словарь.

In [54]:
group_totals = df.groupby('exp_id').agg({'uid':'nunique'}).to_dict()['uid']

In [55]:
group_totals

{246: 2484, 247: 2513, 248: 2537}

### 5.2. A/A тест <a name="5.2"></a>

Для тестирования конверсии между группами будем использовать Z-тест. Поскольку попарных тестирований будет много, напишем функцию.

In [56]:
def z_test(a_hits, a_total, b_hits, b_total):
    """
    функция возвращает p-value по двухстороннему Z-тесту
    """
    # пропорция успехов в группе A
    p1 = a_hits / a_total
    # пропорция успехов в группе B
    p2 = b_hits / b_total
    # комбинированная пропорция успехов
    p = (a_hits + b_hits) / (a_total + b_total)
    # статистика в ст.отклонениях стандартного нормального распределения
    z_value = (p1 - p2) / np.sqrt(p * (1 - p) * (1/a_total + 1/b_total))
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)
    # находим вероятность отклонения (z_value) от центра распределения
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    return (p_value).round(4)

Проверка работы функции.

In [57]:
z_test(100, 1000, 110, 1000)

0.4657

In [58]:
z_test(100, 1000, 150, 1000)

0.0007

По мере увеличения разброса 'успехов' в двух группах p-value падает, т.е. уменьшается вероятность случайных различий между ними —  функция работает корректно.

Подготовим фрейм, куда бодем добавлять результаты множественных тестов.

In [59]:
multiple_tests = (
    df
    .pivot_table(index='event', columns='exp_id', values='uid', aggfunc='nunique')
    .sort_values(by=248, ascending=False)
)

In [60]:
multiple_tests

exp_id,246,247,248
event,,,
MainScreenAppear,2450,2476,2493
OffersScreenAppear,1542,1520,1531
CartScreenAppear,1266,1238,1230
PaymentScreenSuccessful,1200,1158,1181
Tutorial,278,283,279


**Добавляем результаты двухстороннего A/A-теста по z-критерию во фрейм (он же результат тестирования групп 246 и 247)**.

- Нулевая гипотеза: Конверсии двух выборок по событию **равны**. При этом проверяем каждое событие попарно;
- Альтернативная гипотеза: Конверсии двух выборок по событию **различаются**;
- критерий значимости выбираем стандартный: **α = 0.05**. При этом нужна поправка на множественное тестирование. Будем использовать поправку Бонферрони критерий значимости α / m, где m — количество попарных проверок. **Количество попарных проверок равно 20**: 4 группы (246, 247, 248, а также объединенная группа 246+247) умножить на количество событий 5. Тогда **скорректированный критерий значимости равен 0.05 / 20 = 0.0025**. Таким образом, мы удерживаем вероятность группой ошибки первого рода на уровне критерия значимости (вероятности совершить ошибку первого рода при одном попарном сравнении).

Достаточно отвергнуть нулевую гипотезу хотя бы по одному событию `event`, чтобы сделать вывод о статистически значимой разницы этих выборок.

In [61]:
multiple_tests['246-247 test, p-value'] = z_test(
        multiple_tests[246],
        group_totals[246],
        multiple_tests[247],
        group_totals[247]
    )

In [62]:
multiple_tests

exp_id,246,247,248,"246-247 test, p-value"
event,,,,
MainScreenAppear,2450,2476,2493,0.7571
OffersScreenAppear,1542,1520,1531,0.2481
CartScreenAppear,1266,1238,1230,0.2288
PaymentScreenSuccessful,1200,1158,1181,0.1146
Tutorial,278,283,279,0.9377


**Не удалось отвергнуть нулевую гипотезу о равенстве конверсии ни по одному из 5 событий для групп 246-247**. Ни по одному из попарных 5 тестов для групп 246-247 не получено p-value ниже критерия значимости 0.05 или скоррективного на поправку Бонферрони критерия значимости 0.0025. 

Иными словами, в этих выборках не наблюдается статистистически значимого различия и  A/A-эксперимент можно считать успешным — система разбияния работает корректно.




### 5.3. Тестирование контрольных групп (246, 247) с экспериментальной (248) <a name="5.3"></a>

Создадим объединенную контрольную группу A и переимениуем эксперементируемую группу 248 в группу B.

In [63]:
multiple_tests

exp_id,246,247,248,"246-247 test, p-value"
event,,,,
MainScreenAppear,2450,2476,2493,0.7571
OffersScreenAppear,1542,1520,1531,0.2481
CartScreenAppear,1266,1238,1230,0.2288
PaymentScreenSuccessful,1200,1158,1181,0.1146
Tutorial,278,283,279,0.9377


In [64]:
group_a = multiple_tests[246] + multiple_tests[247]

In [65]:
multiple_tests.insert(2, 'A', group_a)
multiple_tests = multiple_tests.rename(columns={248:'B'})

In [66]:
multiple_tests

exp_id,246,247,A,B,"246-247 test, p-value"
event,,,,,
MainScreenAppear,2450,2476,4926,2493,0.7571
OffersScreenAppear,1542,1520,3062,1531,0.2481
CartScreenAppear,1266,1238,2504,1230,0.2288
PaymentScreenSuccessful,1200,1158,2358,1181,0.1146
Tutorial,278,283,561,279,0.9377


Изменим словарь с количеством пользователей в группах `group_totals`

In [67]:
group_totals['A'] = group_totals[246] + group_totals[247]
group_totals['B'] = group_totals.pop(248)

In [68]:
group_totals

{246: 2484, 247: 2513, 'A': 4997, 'B': 2537}

Добавляем результаты тестов.

In [69]:
multiple_tests['246-B test, p-value'] = z_test(
        multiple_tests[246],
        group_totals[246],
        multiple_tests['B'],
        group_totals['B']
    )

multiple_tests['247-B test, p-value'] = z_test(
        multiple_tests[247],
        group_totals[247],
        multiple_tests['B'],
        group_totals['B']
    )

multiple_tests['A-B test, p-value'] = z_test(
        multiple_tests['A'],
        group_totals['A'],
        multiple_tests['B'],
        group_totals['B']
    )

In [70]:
multiple_tests

exp_id,246,247,A,B,"246-247 test, p-value","246-B test, p-value","247-B test, p-value","A-B test, p-value"
event,,,,,,,,
MainScreenAppear,2450,2476,4926,2493,0.7571,0.2950,0.4587,0.2942
OffersScreenAppear,1542,1520,3062,1531,0.2481,0.2084,0.9198,0.4343
CartScreenAppear,1266,1238,2504,1230,0.2288,0.0784,0.5786,0.1818
PaymentScreenSuccessful,1200,1158,2358,1181,0.1146,0.2123,0.7373,0.6004
Tutorial,278,283,561,279,0.9377,0.8264,0.7653,0.7649


**В результате A/B теста не удалось отвергнуть нулевую гипотезу. Не наблюдаем статистически значимой разницы в конверсиях группы A и B. Если эксперимент идет достаточно долго, тест можно останавливать, а изменения в B-версии продукта считать незначительными.**

При этом отдельные 246-B и 247-B тесты показывают достаточно далекие друг от друга результаты. В 246-B тесте по любому из конверсий событий p-value меньше, чем в тесте 247-B. Однако в любом из этих тестов мы все так же не можем отвергнуть нулевую гипотезу о равенстве конверсии ни по одному из событий — факт в копилку правильности полученныго вывода по A/B тесту.

###  5.4. Выбранный критерий значимости <a name="5.4"></a>

В качестве критерия значимости мы выбрали **α = 0.05**. Из-за поправок Бонферрони на множественные проверки скорректированный критерий значимости равен **0.05 / 20 = 0.0025**. При этом ни одна из попарных проверок не дала p-value ниже даже нескоррективного уровня **α = 0.05**, не говоря уже об уровне **0.0025**, т.е. не удалось отвергнуть ни одну нулевую гипотезу.

В реальности поправка на 20 попарных проверок — это слишком много. Хотя мы и держим вероятность ошибки первого рода ниже заданного критерия значимости, мы сильно уменьшаем мощность теста, т.е. увеличиваем вероятность ошибки второго рода — вероятность ошибочно принять неверную нулевую гипотезу, то есть не найти различий в выборках там, где они на самом деле есть. Эта проблема характерна и для других поправок.

Мы могли бы использовать более высокий критерий значимости **α = 0.10**, чтобы немного увеличить мощность теста, но на наших данных это не изменило бы результатов.

## 6. Результаты A/A/B-теста <a name="6"></a>

- A/A-тест не показал статистически значимых различий между двумя контрольными группами: 246 и 247. Значит считаем, что система разбиения пользователей по группам работает корректно;
- результат A/B-теста на объединенных контрольных группах также не показал разницы между группами A (246+247) и B (248). Если эксперимент идет достаточно долго, тест можно останавливать, а изменения в B-версии продукта считать слишком незначительными для внедрения.